###### Setting up Environment variables

In [1]:
import sys
sys.path.append('/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/')
print(sys.version)

3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]


In [2]:
###########import necessary keys 

import os
#import openai
from secret import OPENAI_API_TYPE,OPENAI_API_KEY,OPENAI_API_BASE,HUGGINGFACEHUB_API_TOKEN
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# openai.api_version = "2023-07-01-preview"

os.environ['OPENAI_API_TYPE']=OPENAI_API_TYPE
os.environ['OPENAI_API_BASE']=OPENAI_API_BASE
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN






###### Generate sample data for AUM

In [3]:
import pandas as pd
import numpy as np
import random
#### generate sample data for testing
# 1. Investor Information
investors_data = {
    'investor_id': [i for i in range(1, 51)],
    'investor_name': ['Investor ' + str(i) for i in range(1, 51)],
    'contact_info': ['investor' + str(i) + '@example.com' for i in range(1, 51)],
    'address': ['Address ' + str(i) for i in range(1, 51)],
    'account_open_date': pd.date_range(start='2019-01-01', periods=50, freq='15D').strftime('%Y-%m-%d').tolist()
}
investors_df = pd.DataFrame(investors_data)
print("investors_df")
# 2. Asset Information
assets_data = {
    'asset_id': [i for i in range(1, 201)],
    'investor_id': [random.choice(range(1, 51)) for _ in range(200)],
    'asset_type': random.choices(['stocks', 'bonds', 'real estate', 'mutual fund'], k=200),
    'asset_name': ['Asset ' + str(i) for i in range(1, 201)],
    'purchase_date': pd.date_range(start='2019-01-01', periods=200, freq='3D').strftime('%Y-%m-%d').tolist(),
    'purchase_price': [round(random.uniform(1000, 10000), 2) for _ in range(200)],
    'current_value': [round(random.uniform(1000, 10000), 2) for _ in range(200)],
    'currency': ['USD' for _ in range(200)]
}
assets_df = pd.DataFrame(assets_data)
print("assets_df")

# 3. Transaction Information
transactions_data = {
    'transaction_id': [i for i in range(1, 500001)],
    'investor_id': [random.choice(range(1, 51)) for _ in range(500000)],
    'asset_id': [random.choice(range(1, 201)) for _ in range(500000)],
    'transaction_type': random.choices(['buy', 'sell'], k=500000),
    'transaction_date': pd.date_range(start='2019-01-01', periods=500000, freq='H').strftime('%Y-%m-%d').tolist(),
    'transaction_amount': [round(random.uniform(1, 100), 2) for _ in range(500000)],
    'transaction_price': [round(random.uniform(100, 1000), 2) for _ in range(500000)]
}
transactions_df = pd.DataFrame(transactions_data)
print("transactions_df")

investors_df
assets_df
transactions_df


In [130]:
assets_df.columns

Index(['asset_id', 'investor_id', 'asset_type', 'asset_name', 'purchase_date',
       'purchase_price', 'current_value', 'currency'],
      dtype='object')

In [8]:
##### test if azureopen is working
### deploy 1 (Successful setting)
#os.environ['OPENAI_API_VERSION'] = "2022-12-01"
#deploy_name = "deploy-gpt35turbo16k"
#mod_name = "gpt-35-turbo-16k"

### deploy 2
#os.environ['OPENAI_API_VERSION'] = "2022-12-01"
#deploy_name = "datachatdeploy"
#mod_name = "gpt-35-turbo"


### deploy 3
#os.environ['OPENAI_API_VERSION'] = "2022-12-01"
#deploy_name = "deploy-name-embed"
#mod_name = "text-embedding-ada-002"


#del llm_a
#del llm_o
#del llm_oc


#from langchain.chat_models import AzureChatOpenAI
#llm_a = AzureOpenAI(deployment_name=deploy_name,model_name=mod_name)
#llm_o = OpenAI(engine=deploy_name,model_name=mod_name)
#llm_oc = OpenAIChat(engine=deploy_name,model_name=mod_name) #(Successful setting)



#print(llm_a)
#print(llm_o)
#print(llm_oc)
#llm("what is the role of ESG in asset management")
#llm_oc("what is esg?")

###### Set & Identify right LLM Model & Langchain Module 

In [4]:
##### Unstable connectivity between langchain and openai. Better to test which link is working an any given point in time. 

os.environ['OPENAI_API_VERSION'] = "2023-08-01-preview" #"2023-05-15" #"2022-12-01"
deployment_list =["deploy-name-embed","deploy-gpt35turbo16k", "datachatdeploy"]
model_list=["text-embedding-ada-002","gpt-35-turbo-16k", "text-davinci-003","gpt-35-turbo"]

for deploy_name in deployment_list:
    for mod_name in model_list:
        try:
            llm = AzureOpenAI(deployment_name=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : AzureOpenAI")
                break
        except Exception as e:
            print(f"Error with AzureOpenAI:{e}\n" )
        
        try:
            llm = AzureChatOpenAI(deployment_name=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : AzureOpenAIChat")
                break
        except Exception as e:
            print(f"Error with AzureOpenAIChat:{e}\n" )
        
        try:
            llm= OpenAI(engine=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : OpenAI")
                break
        except Exception as e:
            print(f"Error with OpenAI:{e}\n" )

        try:
            llm= OpenAIChat(engine=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : OpenAIChat")
                break
        except Exception as e:
            print(f"Error with OpenAIChat:{e}\n" )




AzureOpenAI
Params: {'deployment_name': 'deploy-name-embed', 'model_name': 'text-embedding-ada-002', 'temperature': 0.0, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}
Error with AzureOpenAI:The completion operation does not work with the specified model, text-embedding-ada-002. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='text-embedding-ada-002' temperature=0.0 model_kwargs={} openai_api_key='ff0b3bcd1b9843e2bf0c172b531cb52a' openai_api_base='https://ws-openai-ml-aoai.openai.azure.com/' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=No

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/utils/utils.py:157: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
print(llm)

OpenAIChat
Params: {'model_name': 'text-embedding-ada-002', 'engine': 'datachatdeploy', 'temperature': 0}


In [6]:
llm("what is esg?") 

"ESG stands for Environmental, Social, and Governance. It is a framework used to evaluate the sustainability and ethical impact of a company or investment. \n\nEnvironmental factors refer to a company's impact on the environment, such as its carbon emissions, waste management, and resource usage. Social factors consider a company's impact on society, including its treatment of employees, community engagement, and diversity and inclusion practices. Governance factors assess the company's leadership, board structure, executive compensation, and transparency in decision-making.\n\nESG criteria are used by investors, financial institutions, and stakeholders to assess the long-term sustainability and ethical practices of a company. It helps them make informed decisions about investments, as well as encourages companies to adopt responsible business practices."

In [7]:
from langchain.agents import create_pandas_dataframe_agent
import pandas as pd
#langchain.debug = True

agent = create_pandas_dataframe_agent(llm, assets_df, verbose=True,handle_parsing_errors="Check your output and make sure it conforms!")
#"Check your output and make sure it conforms! Do not output an action and a final answer at the same time.")


In [ ]:
agent.run("what is the total aum?")

In [20]:
query = "what is the AUM by year?"
#query = query + " using tool python_repl_ast"
agent.run(input=query)



> Entering new AgentExecutor chain...
Thought: To calculate the AUM (Assets Under Management) by year, we need to group the data by year and then sum the current values of the assets in each year.
Action: Use the `groupby` function to group the data by year and then use the `sum` function to calculate the sum of the current values.
Action Input: `df.groupby(df['purchase_date'].dt.year)['current_value'].sum()`
Observation: Use the `groupby` function to group the data by year and then use the `sum` function to calculate the sum of the current values. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the code.
Action: python_repl_ast
Action Input: `df.groupby(df['purchase_date'].dt.year)['current_value'].sum()`
Observation: AttributeError: Can only use .dt accessor with datetimelike values
Thought:The `purchase_date` column is not recognized as a datetime column. I need to convert it to a datetime format before using the `dt` 

'The AUM by year is $721,932.88 for 2019 and $417,901.19 for 2020.'

##### Attempt to modify Agent Prompt Template (Works with Basic queries but complex queries fails.)

In [23]:
agent.agent.llm_chain.prompt.template = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.

You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be `python_repl_ast`
Action Input: the input to the action
Observation: the result of the action
...the Thought/Action/Action Input/Observation can repeat N times
Thought: I now know the answer to the question
Final Answer: the final answer to the original input question

This is the result of `print(df.head())`:
{df_head}

IMPORTANT: Every <Thought:> must either come with an <Action: and Action Input:> or <Final Answer:>

Begin!
Question: {input}
{agent_scratchpad}
"""

In [41]:
query = "which are the top 5 clients based on the total AUM"
query = query + " using tool python_repl_ast"
agent.run(input=query)



> Entering new AgentExecutor chain...
Thought: I need to calculate the total AUM for each client and then sort them in descending order to find the top 5 clients.

Action: python_repl_ast
Action Input: df.groupby('investor_id')['current_value'].sum().sort_values(ascending=False).head(5)

Observation: investor_id
37    46877.58
4     43246.46
17    41405.91
9     37534.67
44    35626.54
Name: current_value, dtype: float64
Thought:I have the total AUM for the top 5 clients. Now I need to find their names.

Action: python_repl_ast
Action Input: df[df['investor_id'].isin([37, 4, 17, 9, 44])]['investor_id', 'asset_name']

Observation: KeyError: ('investor_id', 'asset_name')
Thought:There seems to be an error in the code. I mistakenly used square brackets instead of curly braces to select multiple columns. I will correct it and try again.

Action: python_repl_ast
Action Input: df[df['investor_id'].isin([37, 4, 17, 9, 44])][['investor_id', 'asset_name']]

Observation:      investor_id asset

'Agent stopped due to iteration limit or time limit.'

In [72]:
###### PREFIX & SUFFIX for template. (underlying library is not accepting SUFFIX)

In [71]:


PREFIX = '''
You are working with a pandas dataframe in Python that contains Asset Under Management or AUM Data for Clients. The name of the dataframe are  `df`

The data includes clients investment portfolio across various asset types and transactional information about buy and sell of assets. You are an expert in Asset Under Management or AUM and know details about asset management.

Keep in mind that columns names maybe different from input i.e Column name is "investor_name" but input could be "client" or "Client Name" etc. 

You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.Necessary library pandas is already install.
'''

SUFFIX = '''
Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

This is the result of `print(df.head())`:
{df}

Begin!

Question: {input}
{agent_scratchpad}
'''

In [73]:

#agent = create_pandas_dataframe_agent(OpenAIChat(engine="deploy-gpt35turbo16k",model_name="gpt-35-turbo-16k",temperature=0), df, verbose=True,prefix=PREFIX,suffix=SUFFIX) #, suffix=SUFFIX
agent = create_pandas_dataframe_agent(OpenAIChat(engine="deploy-gpt35turbo16k",model_name="gpt-35-turbo-16k",temperature=0), df, verbose=True,prefix=PREFIX) #, suffix=SUFFIX

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
class NewAgentOutputParser(BaseOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Any:
        print("-" * 20)
        cleaned_output = text.strip()
        # Regex patterns to match action and action_input
        action_pattern = r'"action":\s*"([^"]*)"'
        action_input_pattern = r'"action_input":\s*"([^"]*)"'

        # Extracting first action and action_input values
        action = re.search(action_pattern, cleaned_output)
        action_input = re.search(action_input_pattern, cleaned_output)

        if action:
            action_value = action.group(1)
            print(f"First Action: {action_value}")
        else:
            print("Action not found")

        if action_input:
            action_input_value = action_input.group(1)
            print(f"First Action Input: {action_input_value}")
        else:
            print("Action Input not found")

        print("-" * 20)
        if action_value and action_input_value:
            return {"action": action_value, "action_input": action_input_value}

        # Problematic code left just in case
        if "```json" in cleaned_output:
            _, cleaned_output = cleaned_output.split("```json")
        if "```" in cleaned_output:
            cleaned_output, _ = cleaned_output.split("```")
        if cleaned_output.startswith("```json"):
            cleaned_output = cleaned_output[len("```json"):]
        if cleaned_output.startswith("```"):
            cleaned_output = cleaned_output[len("```"):]
        if cleaned_output.endswith("```"):
            cleaned_output = cleaned_output[: -len("```")]
        cleaned_output = cleaned_output.strip()
        response = json.loads(cleaned_output)
        return {"action": response["action"], "action_input": response["action_input"]}
        # end of problematic code

def make_chain():
    memory = ConversationBufferMemory(
        memory_key="chat_history", return_messages=True)

    agent = ConversationalChatAgent.from_llm_and_tools(
        llm=ChatOpenAI(), tools=[], system_message=SYSTEM_MESSAGE, memory=memory, verbose=True, output_parser=NewAgentOutputParser())

    agent_chain = AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True,
    )
    return agent_chain

##### adding memory to the agent (Have not yet tested if memory works)

In [38]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [43]:
agent = create_pandas_dataframe_agent(llm, assets_df, verbose=True,memory=memory,handle_parsing_errors="Check your output and make sure it conforms!")

In [45]:
query = "calculate AUM for Investor 10"
query = query + " using tool python_repl_ast"
agent.run(input=query)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the investor_id is 10, and then sum the current_value column.

Action: python_repl_ast
Action Input: df[df['investor_id'] == 10]['current_value'].sum()
Observation: 35135.06
Thought:The AUM for Investor 10 is 35135.06
Final Answer: 35135.06

> Finished chain.


'35135.06'

In [46]:
agent.run("Identify top 10 Investors based on AUM")



> Entering new AgentExecutor chain...
Thought: To identify the top 10 investors based on AUM (Assets Under Management), we need to calculate the total current value of assets for each investor and then sort them in descending order.

Action: We can use the `groupby` function to group the dataframe by investor_id and then calculate the sum of the current_value for each group. Then we can sort the resulting dataframe by the sum of current_value in descending order.

Action Input: 
```python_repl_ast
df.groupby('investor_id')['current_value'].sum().sort_values(ascending=False).head(10)
```

Observation: We can use the `groupby` function to group the dataframe by investor_id and then calculate the sum of the current_value for each group. Then we can sort the resulting dataframe by the sum of current_value in descending order. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the command and see the result.
Action: python_repl_a

'The top 10 investors based on AUM are as listed above.'

In [50]:
agent.run("Identify top 10 Investors based on AUM")



> Entering new AgentExecutor chain...
Thought: To identify the top 10 investors based on AUM (Assets Under Management), we need to calculate the total current value of assets for each investor and then sort them in descending order.

Action: I will group the dataframe by 'investor_id' and calculate the sum of 'current_value' for each group.

Action Input: `df.groupby('investor_id')['current_value'].sum()`

Observation: I will group the dataframe by 'investor_id' and calculate the sum of 'current_value' for each group. is not a valid tool, try one of [python_repl_ast].
Thought:

OutputParserException: Could not parse LLM output: `I need to use the `python_repl_ast` tool to execute the action.`

### Create Python Agent with PythonREPLTool (Basic REPL Tool works but complex queries fails)


In [7]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool


In [11]:
print(llm)

OpenAIChat
Params: {'model_name': 'text-embedding-ada-002', 'engine': 'deploy-gpt35turbo16k', 'temperature': 0}


In [12]:
agent_exec = create_python_agent(llm=llm,tool=PythonREPLTool(),verbose=True)

In [24]:
import numpy
agent_exec.run(f"column names of the dataframe 'df'".format(assets_df))



> Entering new AgentExecutor chain...
I need to access the dataframe 'df' and retrieve its column names.
Action: Python_REPL
Action Input: df.columns
Observation: NameError("name 'df' is not defined")
Thought:I need to define the dataframe 'df' before I can access its column names.
Action: Python_REPL
Action Input: df = pd.DataFrame()
Observation: NameError("name 'pd' is not defined")
Thought:I need to import the pandas library before I can create a dataframe.
Action: Python_REPL
Action Input: import pandas as pd
Observation: 
Thought:I have imported the pandas library as 'pd'. Now I can create a dataframe.
Action: Python_REPL
Action Input: df = pd.DataFrame()
Observation: 
Thought:I have created the dataframe 'df'. Now I can access its column names.
Action: Python_REPL
Action Input: df.columns
Observation: 
Thought:I now know the final answer
Final Answer: []

> Finished chain.


'[]'

### plan and execute agents testing (Not tested Yet)

In [1]:
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner


#### Custom Agent for passing complex Query (WIP)

In [10]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool,PythonAstREPLTool
from langchain.memory import ConversationBufferMemory


In [55]:
python = PythonAstREPLTool(locals={"assets_df": assets_df}) # set access of python_repl tool to the dataframe
assets_df_columns = assets_df.columns.to_list() # print column names of df

In [56]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [

    Tool(
        name = "Investor Client Asset Data",
        func=python.run,
        description = f"""
        Useful for when you need to answer questions about investor client asset portfolio data stored in pandas dataframe 'assets_df'. 
        Run python pandas operations on 'assets_df' to help you get the right answer.
        'assets_df' has the following columns: {assets_df_columns}
        
        <user>: what is the AUM of client 10?
        <assistant>:  assets_df[assets_df['investor_id'] == 10]['current_value'].sum()
        <assistant>: client or investor 1 has total AUM n.  

        <user>: what is the AUM of client 10 by asset type?
        <assistant>:  assets_df[assets_df['investor_id'] == 10].groupby("asset_type")['current_value'].sum()
        <assistant>: client or investor 1 has total AUM by asset type n.   

        <user>: what is the AUM of client 10 by asset type and year?
        <assistant>:  assets_df[assets_df['investor_id'] == 5].groupby(['asset_type', assets_df['purchase_date'].dt.year])['current_value'].sum()
        <assistant>: client or investor 1 has total AUM by asset type and year n.            
        """
    )
]


In [20]:
#assets_df[assets_df['investor_id'] == 10].groupby("asset_type")['current_value'].sum()

asset_type
bonds          1420.09
mutual fund    6318.62
Name: current_value, dtype: float64

In [76]:

# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
## Added first
agent_kwargs = {'prefix': f'You are Asset Under Management Expert. You are tasked to assist on questions related to client or investor AUM. You have access to the following tools:'}
### Added second

#memory = ConversationBufferMemory(memory_key="chat_history")


# initialize the LLM agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                        # memory=memory, 
                         agent_kwargs=agent_kwargs
                         )

In [84]:
query0="Hi, I am AK."
query1=" what is the AUM of client 5"
query2=" what is the AUM of Investor 10 by asset type"
query3=" what is the AUM of client 5 by asset type and year"
query4="which are the top 5 clients for bonds.Show their AUM for bonds"
query5="what is the AUM of Investor 6 for 2019 in Bonds"


In [ ]:
agent.run(query0) # OutputParserException: Could not parse LLM output: `I need to use the Investor Client Asset Data tool to assist AK.`

In [72]:
agent.run("what is the AUM of Investor 6 by asset type")



> Entering new AgentExecutor chain...
I need to filter the data for Investor 6 and then group it by asset type to calculate the AUM.
Action: Investor Client Asset Data
Action Input: assets_df[assets_df['investor_id'] == 6].groupby("asset_type")['current_value'].sum()
Observation: asset_type
bonds           3461.86
mutual fund    23360.93
Name: current_value, dtype: float64
Thought:The AUM of Investor 6 by asset type is as follows:
- Bonds: $3,461.86
- Mutual Fund: $23,360.93
Thought: I now know the final answer
Final Answer: The AUM of Investor 6 by asset type is $3,461.86 for bonds and $23,360.93 for mutual funds.

> Finished chain.


'The AUM of Investor 6 by asset type is $3,461.86 for bonds and $23,360.93 for mutual funds.'

In [74]:
agent.run("what is the AUM of Investor 6 by asset type and year")



> Entering new AgentExecutor chain...
I need to filter the data for Investor 6 and group it by asset type and year.
Action: Investor Client Asset Data
Action Input: assets_df[assets_df['investor_id'] == 6].groupby(['asset_type', assets_df['purchase_date'].dt.year])['current_value'].sum()
Observation: asset_type   purchase_date
bonds        2019              3461.86
mutual fund  2019             23360.93
Name: current_value, dtype: float64
Thought:I now know the final answer
Final Answer: The AUM of Investor 6 by asset type and year is as follows:
- Bonds in 2019: $3,461.86
- Mutual Fund in 2019: $23,360.93

> Finished chain.


'The AUM of Investor 6 by asset type and year is as follows:\n- Bonds in 2019: $3,461.86\n- Mutual Fund in 2019: $23,360.93'

In [95]:
agent.run("what is the AUM of Investor 6 for Bonds in 2019")



> Entering new AgentExecutor chain...
I need to filter the data for Investor 6, Bonds, and the year 2019. Then I can calculate the AUM by summing the current values.
Action: Investor Client Asset Data
Action Input: assets_df[assets_df['investor_id'] == 6]
Observation:     asset_id  investor_id   asset_type asset_name purchase_date  \
29        30            6  mutual fund   Asset 30    2019-03-29   
57        58            6  mutual fund   Asset 58    2019-06-21   
96        97            6        bonds   Asset 97    2019-10-16   
99       100            6  mutual fund  Asset 100    2019-10-25   

    purchase_price  current_value currency  
29         4576.15        5800.11      USD  
57         7893.73        9377.47      USD  
96         1038.08        3461.86      USD  
99         1509.40        8183.35      USD  
Thought:I need to filter the data for Investor 6, Bonds, and the year 2019. Then I can calculate the AUM by summing the current values.
Action: Investor Client Asset Da

'3461.86 USD'